# Training a sentiment analysis model

In [315]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import movie_reviews, stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
import time

In [316]:
reviews = [[file.split("/")[0], file] for file in movie_reviews.fileids()]
df = pd.DataFrame(reviews, columns=["label", "fileid"])

In [317]:
stopSet = set(stopwords.words("english"))

In [353]:
punc = set(list(string.punctuation) + ["\n", ""])
cleanword = lambda word: (word not in punc) and (not word.isnumeric())
def preprocess(words):
    clean_mess = [w for w in words if w not in stopSet and cleanword(w)]
    return " ".join(clean_mess)


In [348]:
print(len(movie_reviews.words("neg/cv000_29416.txt")))
len(preprocess(movie_reviews.words("neg/cv000_29416.txt")))

879


2545

## Evaluating pretrained model

In [349]:
sia = SentimentIntensityAnalyzer()

In [350]:
sn = " ".join(tokenize(example))
sia.polarity_scores(sn)

{'neg': 0.162, 'neu': 0.639, 'pos': 0.199, 'compound': 0.9209}

In [351]:
df['compound'] = df.fileid.apply(lambda x: sia.polarity_scores(preprocess(movie_reviews.words(x)))['compound'])

In [352]:
df.groupby("label").describe().T

label                   neg          pos
compound count  1000.000000  1000.000000
         mean      0.228477     0.654830
         std       0.876383     0.688840
         min      -0.999000    -0.999000
         25%      -0.892100     0.918475
         50%       0.869000     0.988800
         75%       0.982000     0.996500
         max       0.999100     0.999800

In [327]:
df.groupby("label").describe().T

label                   neg          pos
compound count  1000.000000  1000.000000
         mean      0.107110     0.646057
         std       0.904130     0.702314
         min      -0.999700    -0.999600
         25%      -0.954550     0.902900
         50%       0.682050     0.990900
         75%       0.982025     0.997225
         max       0.999600     0.999900